In [1]:
#imports
import numpy as np
import scipy.stats as stats
import os

### Feature extraction
train will have the training samples and test will have the test samples

In [6]:
#speaker dependent partitions
#sdeptrain dimensions sdeptrain[speaker][word][utterance idx] with idxs given alphabetically
sdeptrain = [[[[] for i in range(4)] for j in range(5)] for k in range(4)]
#sdeptest dimensions sdeptrain[speaker][word] with idxs given alphabetically
sdeptest = [[[] for i in range(5)] for j in range(4)]
p = './feature'
speakerList = ['dg', 'ls', 'mh', 'yx']
wordList = ['asr','cnn','dnn','hmm','tts']
snum = 0
for s in speakerList:
    speakerpath = p+'/'+s
    wnum = 0
    for w in wordList:
        wordpath = speakerpath +'/'+s+'_'+w
        #set up train
        for utt in range(1,5):
            fpath = wordpath + str(utt)+'.fea'
            f = open(fpath,'r')
            data = f.readlines()
            data = [i.split(',') for i in data]
            for sample in range(len(data)):
                data[sample] = [float(i) for i in data[sample]]
            for d in data:
                sdeptrain[snum][wnum][utt-1].append(d)
        #set up test
        fpath2 = wordpath + str(5)+'.fea'
        f2 = open(fpath2,'r')
        data = f2.readlines()
        data = [i.split(',') for i in data]
        for d in data:
            sdeptest[snum][wnum].append(d)
        wnum += 1
    snum += 1
sdeptrain = np.array(sdeptrain)

In [8]:
# print(sdeptrain[0][0])

In [9]:
#speaker independent partitions
#Extract feature vectors
p = './feature'
#test separetes each utterance of each word of each speaker
test = []
train = []
#test_big chunks all utterances of each word for each speaker
test_big = []
train_big = []
dirList = ['dg', 'ls', 'mh', 'yx']
for d in os.listdir(p):
    #put mh in the test set
    if d == 'mh':
        dirpath = p + '/' + d
        filelist = os.listdir(dirpath)
        word = [[] for i in range(5)]
        word_big = [[] for i in range(5)]
        for i in range(len(filelist)):
            f = open(dirpath + '/' + filelist[i])
            data_ = f.readlines() 
            data = [i.split(',') for i in data_]
            for sample in range(len(data)):
                data[sample] = [float(s) for s in data[sample] ]
            word[i//5].append(data)
            word_big[i//5].extend(data)
        test.append(word)
        test_big.append(word_big)
    else:
        if d in dirList:
            #put all others in train
            dirpath = p + '/' + d
            filelist = os.listdir(dirpath)
            word = [[] for i in range(5)]
            word_big = [[] for i in range(5)]
            for i in range(len(filelist)):
                f = open(dirpath + '/' + filelist[i])
                data_ = f.readlines() 
                data = [i.split(',') for i in data_]
                for sample in range(len(data)):
                    data[sample] = [float(s) for s in data[sample] ]
                word[i//5].append(data)
                word_big[i//5].extend(data)
            train.append(word)
            train_big.append(word_big)

In [10]:
print(len(train_big),len(train_big[0]))

3 5


In [13]:
#speaker dependent hmm

print(np.exp(1))

2.71828182846


In [6]:
a = [[1,1,1],
     [2,2,2],
     [3,3,3]]

a_ = [a[-i] for i in range(1,len(a)+1)]
print(a_)

[[3, 3, 3], [2, 2, 2], [1, 1, 1]]


In [9]:
##### HELPER FUNCTIONSSSSS
#implemented from https://en.wikipedia.org/wiki/Baum%E2%80%93Welch_algorithm

#Forward Pass

def calculateBMatrix(x, mu, sigma, n):
    #n denotes which state we are in {1...N} N = 5
    #x is a single time frame (1x14)
    #mu is the average across all states 
    #sigma is the covariance matrix across all states 
    
    #BMatrix = [T,N]

    
    bMatrix = []
    for frame in range(T):
        temp_frame = []
        for state in range(N):
            temp_frame.append(stats.multivariate_normal(mu[i],sigma[i]).pdf(X[t]))
        bMatrix.extend(temp_frame)
        
    return bMatrix

def generateAlphaMatrix(transMatrix, BMatrix):
    #populate our alpha matrix
    #each utterance has an alpha matrix?
    
    #alphaMatrix = [T,N]
    
    alphaMatrix = []
    T = len(BMatrix)
    N = len(BMatrix[0])
    for frame in range(T):
        temp_frame = []
        for state in range(N):
            if frame == 0:
                #if we are in the first frame
                temp_frame.append(priors[state] * BMatrix[frame][state])
            else:
                #any other frame
                s = sum([ alphaMatrix[frame-1][n] * transMatrix[n][state] for n in range(N)])
                x = BMatrix[frame][state] * s
                temp_frame.append(x)
        alphaMatrix.extend(temp_frame)
        
    return alphaMatrix
       
#Backward Pass

def calculateBetaMatrix(transMatrix, bMatrix):
    #populate the Beta matrix
    #create a beta matrix for each file?
    
    betaMatrix = []
    T = len(bMatrix)
    N = len(bMatrix[0])
    
    #make this beta matrix backwards
    for frame in range(T):
        temp_frame = []
        for state in range(N):
            if frame == 0:
                temp_frame.append(1)
            else:
                s = sum([ BetaMatrix[frame-1][n] * transMatrix[state][n] * BMatrix[frame-1][n]  for n in range(N)])
                temp_frame.append(s)
        betaMatrix.extend(temp_frame)
        
    #reverse the matrix
    betaMatrix_ = [betaMatrix[-i] for i in range(1, T+1)]
    return betaMatrix_
    
#Update step helper functions

def calculateGammaMatrix(alphaMatrix, betaMatrix):
    #denominator should be the same as xi denominator
    T = len(betaMatrix)
    N = len(betaMatrix[0])
    
    sumList = [sum([alphaMatrix[f][n] * betaMatrix[f][n] for n in range(N)]) for f in range(T)]
    
    gammaMatrix = []
    for frame in range(T):
        temp_frame = []
        for state in range(N):
            gammaMatrix[frame][state] = alphaMatrix[frame][state] / sumList[frame]
    return gammaMatrix

def calculateXi( transMatrix, alphaMatrix, betaMatrix, bMatrix):
    #caluculate xi 
    #probability of going from state n to state j at times t and t+1 respectively
    T = len(bMatrix)
    N = len(bMatrix[0])
    
    xiMatrix = [[0 for i in range(2*N)] for j in range(T)]
    
    for frame in range(T):
        for state in range(N):
            for state_ in range(state, state + 2):
                xiMatrix[frame, state + state_] = alphaMatrix[frame, state] * transMatrix[state][state_]
                if frame < (T-1):
                    if state_ == N:
                        xiMatrix[frame, state + state_] = 0
                    else:
                        xiMatrix[frame, state + state_] = xiMatrix[frame, state + state_] * bMatrix[frame + 1, state_] * betaMatrix[frame+1, state_]
        xi[:,t]=xi[:,t]/np.sum(xi[:,t])
        xiMatrix[frame,:] = xiMatrix[frame,:]/np.sum(xiMatrix[frame, :]) 
    
    return xiMatrix 

#Update steps
def updatePriors(gammaMatrix):
    #we want the expected frequency spent in state i at time step t = 1
    new_priors = []
    N = len(gammaMatrix[0])
    for n in range(N):
        new_priors.append(gammaMatrix[0][n])
    return new_priors

def updateA(priors, transMatrix, X, mu, sigma, n, j, t, N, T):
    num = 0
    for i in range(T-1):
        #across all t
        temp = calculateXi(transMatrix, X, mu, sigma, n, j, i, N, T)
        num += temp
        
    denom = 0
    for i in range(T-1):
        #across all t
        temp = calculateGamma(priors, transMatrix, X, mu, sigma, n, i, N, T)
        denom += temp
    
    alpha_update = num / denom
    
    return alpha_update

def updateB(vk, priors, transMatrix, X, mu, sigma, n, N, T):
    #vk is a specific observation
    #how to compare that against all the y_t?
    num = 0
    for i in range(T):
        #if yt == vk:
        temp = calculateGamma(priors, transMatrix, X, mu, sigma, n, i, N, T)
        num += temp
    denom = 0
    for i in range(T):
        temp = calculateGamma(priors, transMatrix, X, mu, sigma, n, i, N, T)
        denom += temp
    b_update = num / denom
    
    return b_update

In [11]:
#speaker independent hmm
initialProbabilities = [0.2, 0.2, 0.2, 0.2, 0.2]

initialTransition = [[0.8,0.2,  0,  0,   0],
                     [  0,0.8,0.2,  0,   0],
                     [  0,  0,0.8,0.2,   0],
                     [  0,  0,  0,0.8, 0.2],
                     [  0,  0,  0,  0,   1]]

#create an HMM for each word
word0 = []
word1 = []
word2 = []
word3 = []
word4 = []

for speaker in range(len(train_big)):
    for word in range(len(train_big[speaker])):
        if word == 0:
            word0.extend(train_big[speaker][word])
        if word == 1:
            word1.extend(train_big[speaker][word])
        if word == 2:
            word2.extend(train_big[speaker][word])
        if word == 3:
            word3.extend(train_big[speaker][word])
        if word == 4:
            word4.extend(train_big[speaker][word])

#initialize mean of hmm for the first word
word0 = np.array(word0)
#mu should be (1x14) for each state
# mu_matrix
mu0 = np.mean(word0, axis=0)
#initialize the covariance matrix, sigma, of the hmm for the first word
word0_np = np.array(word0)
#covmat should be (14x14) for each state
covmat0 = np.cov(word0_np.T)

# Baum Welch this bitchhhhhh



Comments

There are N = 5 possible states and the number of observations is equal to the number of rows or time steps for each utterance. At each time step or observation, we are in one of the states, so a number 1-5 i think. So T = how many ever rows there are present. Still not sure exactly how we should chunk the data. Not sure if we should separate it by utterance or if we should group all the utterances of the same word together. Update parameters using baum welch forward/backward propogation equations. According to Piazza, update alpha and beta after each individual utterance and update the transition matrix after all the files for that word? Not how to implement this though. 

We train one hmm per word, so 5 in total. To predict a test sample, we run the utterance through each of the 5 models and assign it the label of the hmm that gives the highest probability.

In [112]:
# def covmat(data):
#     #create the covariance matrix
#     #each row is a variable
#     #each column is an observation of each of those variables
    
    
    

[-2.8224, -14.252, 1.6086, -0.41555, 0.055411, 0.20185, 0.41501, 0.10763, -0.033127, -0.26792, 0.0079, -0.023143, 0.30929, 0.25359]
